In [2]:
airlinesPath="file:////Users/madsh01/Personal/Workspace/spark/practice/datasets/FlightsData/airlines.csv"
airportsPath="file:////Users/madsh01/Personal/Workspace/spark/practice/datasets/FlightsData/airports.csv"
flightPath="file:////Users/madsh01/Personal/Workspace/spark/practice/datasets/FlightsData/flights.csv"

In [4]:
airlines = sc.textFile(airlinesPath)

In [5]:
print(airlines)

file:////Users/madsh01/Personal/Workspace/spark/practice/datasets/FlightsData/airlines.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0


In [6]:
airlines.collect()

['Code,Description',
 '"19031","Mackey International Inc.: MAC"',
 '"19032","Munz Northern Airlines Inc.: XY"',
 '"19033","Cochise Airlines Inc.: COC"',
 '"19034","Golden Gate Airlines Inc.: GSA"',
 '"19035","Aeromech Inc.: RZZ"',
 '"19036","Golden West Airlines Co.: GLW"',
 '"19037","Puerto Rico Intl Airlines: PRN"',
 '"19038","Air America Inc.: STZ"',
 '"19039","Swift Aire Lines Inc.: SWT"',
 '"19040","American Central Airlines: TSF"',
 '"19041","Valdez Airlines: VEZ"',
 '"19042","Southeast Alaska Airlines: WEB"',
 '"19043","Altair Airlines Inc.: AAR"',
 '"19044","Chitina Air Service: CHI"',
 '"19045","Marco Island Airways Inc.: MRC"',
 '"19046","Caribbean Air Services Inc.: OHZ"',
 '"19047","Sundance Airlines: PRO"',
 '"19048","Seair Alaska Airlines Inc.: SAI"',
 '"19049","Southeast Airlines Inc.: SLZ"',
 '"19050","Alaska Aeronautical Indust.: AAZ"',
 '"19051","Imperial Airlines Inc.: IMP"',
 '"19052","Trans Western Airlines Utah: TWA"',
 '"19053","Wright Airlines Inc.: WRT"',
 '"19

In [7]:
airlines.first()

'Code,Description'

In [8]:
airlines.take(10)

['Code,Description',
 '"19031","Mackey International Inc.: MAC"',
 '"19032","Munz Northern Airlines Inc.: XY"',
 '"19033","Cochise Airlines Inc.: COC"',
 '"19034","Golden Gate Airlines Inc.: GSA"',
 '"19035","Aeromech Inc.: RZZ"',
 '"19036","Golden West Airlines Co.: GLW"',
 '"19037","Puerto Rico Intl Airlines: PRN"',
 '"19038","Air America Inc.: STZ"',
 '"19039","Swift Aire Lines Inc.: SWT"']

In [9]:
airlines.count()

1580

In [23]:
airlinesWoHdr= airlines.filter(lambda line : "Description" not in line)

In [12]:
airlinesWoHdr.take(1)

['"19031","Mackey International Inc.: MAC"']

In [13]:
airlinesParsed= airlinesWoHdr.map(lambda line: line.split(","))

In [16]:
airlinesParsed.take(10)

[['"19031"', '"Mackey International Inc.: MAC"'],
 ['"19032"', '"Munz Northern Airlines Inc.: XY"'],
 ['"19033"', '"Cochise Airlines Inc.: COC"'],
 ['"19034"', '"Golden Gate Airlines Inc.: GSA"'],
 ['"19035"', '"Aeromech Inc.: RZZ"'],
 ['"19036"', '"Golden West Airlines Co.: GLW"'],
 ['"19037"', '"Puerto Rico Intl Airlines: PRN"'],
 ['"19038"', '"Air America Inc.: STZ"'],
 ['"19039"', '"Swift Aire Lines Inc.: SWT"'],
 ['"19040"', '"American Central Airlines: TSF"']]

In [33]:
import csv
from io import StringIO


def split(line):
    reader = csv.reader(StringIO(line))
   
    return next(reader)


airlinesWoHdr.map(split).take(10)

[['19031', 'Mackey International Inc.: MAC'],
 ['19032', 'Munz Northern Airlines Inc.: XY'],
 ['19033', 'Cochise Airlines Inc.: COC'],
 ['19034', 'Golden Gate Airlines Inc.: GSA'],
 ['19035', 'Aeromech Inc.: RZZ'],
 ['19036', 'Golden West Airlines Co.: GLW'],
 ['19037', 'Puerto Rico Intl Airlines: PRN'],
 ['19038', 'Air America Inc.: STZ'],
 ['19039', 'Swift Aire Lines Inc.: SWT'],
 ['19040', 'American Central Airlines: TSF']]

In [82]:
from datetime import datetime
from collections import namedtuple


fields = ('date','airline','flightnum', 'origin','dest','dep','dep_delay','arv'
         ,'arv_delay','airtime','distance')

Flight = namedtuple('Flight',fields,verbose=True)

DATE_FMT = "%Y-%m-%d"


TIME_FMT = "%H%M"

def parse(row):
   
    row[0]=datetime.strptime(row[0],DATE_FMT).date()
    row[5]=datetime.strptime(row[5],TIME_FMT).time()
    row[7]=datetime.strptime(row[7],TIME_FMT).time()
    row[6]=float(row[6])
    row[8]=float(row[8])
    row[9]=float(row[9])
    row[10]=float(row[10])
    return Flight(*row[:11])




from builtins import property as _property, tuple as _tuple
from operator import itemgetter as _itemgetter
from collections import OrderedDict

class Flight(tuple):
    'Flight(date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance)'

    __slots__ = ()

    _fields = ('date', 'airline', 'flightnum', 'origin', 'dest', 'dep', 'dep_delay', 'arv', 'arv_delay', 'airtime', 'distance')

    def __new__(_cls, date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance):
        'Create new instance of Flight(date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance)'
        return _tuple.__new__(_cls, (date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance))

    @classmethod
    def _make(cls, iterable, new=tuple.__new__, len=len):
        'Make a new Flight object from a sequence or iterable'
        result = new(cls, iterable)
        if len(result) != 11:
  

In [66]:
flights_rdd = sc.textFile(flightPath)

In [96]:
flightsParsed = flights_rdd.map(lambda item: item.split(",")).map(parse)
flightsParsed.persist()

PythonRDD[57] at RDD at PythonRDD.scala:49

In [85]:
 flightsParsed.first()


Flight(date=datetime.date(2014, 4, 1), airline='19805', flightnum='1', origin='JFK', dest='LAX', dep=datetime.time(8, 54), dep_delay=-6.0, arv=datetime.time(12, 17), arv_delay=2.0, airtime=355.0, distance=2475.0)

In [90]:
total_flights=flightsParsed.count()

In [88]:
totalDistance =flightsParsed.map(lambda obj : obj.distance).reduce(lambda x,y: x+y)

In [91]:
#average distance travelled by all flights
totalDistance/total_flights

794.8585013871385

In [93]:
flightWithDepDelay =flightsParsed.filter(lambda obj: obj.dep_delay>0).count()

In [97]:
#percentage of filghts with departure delay
flightWithDepDelay/total_flights


0.3753871510922012

In [98]:
flightsParsed.unpersist()

PythonRDD[57] at RDD at PythonRDD.scala:49

In [ ]:
#compute average delay
sumCount = flightsParsed.map(lambda obj: obj.dep_delay).aggregate((0,0),
                                                                  (lambda acc, value: (acc[0]+ value, acc[1]+1)),
                                                                  (lambda acc1,acc2: (acc1[0]+acc2[0],acc1[1]+acc2[1] )))